# ETL Proyecto Individual ML Ops STEAM

In [ ]:
# Importo las librerias que considero necesarias para hacer la extración de los Data Set
import json
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Rutas de ubicación para los datasets:
ruta_reviews = 'E:\\AAADATOS\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\australian_user_reviews.json'
ruta_items = 'E:\\AAADATOS\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\australian_users_items.json'
ruta_games = 'E:\\AAADATOS\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\output_steam_games.json'

## Data Set australian_user_reviews.json

In [ ]:
# Se inicializa una lista vacia donde alamcenare los datos del archivo .JSON
filas = []
# Se abre el archivo .JSON con el modulo 'with'
with open(ruta_reviews, 'r', encoding='utf-8') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

In [ ]:
# Se inicializa el primer DataFrame a partir de la lista donde se extrajo el archivo .JSON
df_reviews = pd.DataFrame(filas)
df_reviews

Se evidencia que la columna "reviews" contiene listas como dato así que se tiene que proceder a expandirla

In [ ]:
df_reviews = df_reviews.explode('reviews').reset_index() # Se redefine el DatFrame con una descomposición del campo para desanidarla y se reinicia el conteo del índice
df_reviews = df_reviews.drop(columns='index') # Se elimina el campo 'index'
df_reviews = pd.concat([df_reviews, pd.json_normalize(df_reviews['reviews'])], axis=1) # Se normalizan los datos JSON de la columna 'reviews' aplanando los datos, convirtiendolos en un formato tabular y concatenando de manera horizontal el DatFrame original con el del contendio de 'reviews' 
df_reviews.head()

### Se empieza a analizar y transformar los datos del DataFrame

In [ ]:
# Información general del DataFrame
df_reviews.info()

NULOS

In [ ]:
# Conteo de Nulos
nulos = df_reviews.isna().sum()
nulos

In [ ]:
# Visualizar los valores Nulos
df_nulos = df_reviews[df_reviews.isnull().any(axis=1)] # Se crea un nuevo DatFrame donde se filtra si hay algun valor nulo en la fila a partir del DataFrame orginal
df_nulos

En base al DataFrame anterior se puede afirmar que todos los Nulos corresponden estas filas donde no esta la información relevante para mis objetivos con las consignas que se me asignaron por lo cual se procedera a ser eliminados

In [ ]:
# Elimino las filas con valores nulos del DataFrame con el método 'dropna()'
df_reviews.dropna(inplace=True)
# Conteo de Nulos nuevamente para verificar
nuevos_nulos = df_reviews.isna().sum()
nuevos_nulos

De esta manera se han eliminado los valores Nulos del DataFrame hasta este punto de la transformación

#### Se procede a evaluar el contenido de los campos y determinar su relevancia dentro de los obejtivos

In [ ]:
df_reviews.info()

In [ ]:
# Se crea una variable que contiene los valores unicos del campo 'funny'
valores_funny = df_reviews['funny'].unique() 
valores_funny

In [ ]:
# Se crea otra variable para contar cuantos valores hay que no sean strings vacios ("")
cant_funny = df_reviews[df_reviews['funny'] != ''].count()['funny']
cant_funny

In [ ]:
# Se hace una operación matematica para determinar que porcentaje representa los valores de funny completos dentro del conjunto del DataFrame
total_filas_df = 59305
print(f'La cantidad de valores que no son strings vacios en el campo "funny" representan un {round((cant_funny/total_filas_df)*100, 2)}%')

Después del proceso anterior se puede determinar que los valores que se podrían considerar útiles del campo "funny" apenas representan un 13.74% dentro de todo el DataFrame por lo cual este campo en su conjunto se considerará presindible

In [ ]:
# Se crea una variable que contiene los valores unicos del campo 'last_edited'
valores_last_edited = df_reviews['last_edited'].unique() 
valores_last_edited

In [ ]:
# Se crea otra variable para contar cuantos valores hay que no sean strings vacios ("")
cant_last_edited = df_reviews[df_reviews['last_edited'] != ''].count()['last_edited']
cant_last_edited

In [ ]:
# Se hace una operación matematica para determinar que porcentaje representa los valores de "last_edited" completos dentro del conjunto del DataFrame
print(f'La cantidad de valores que no son strings vacios en el campo "last_edited" representan un {round((cant_last_edited/total_filas_df)*100, 2)}%')

Después del proceso anterior se puede determinar que los valores que se podrían considerar útiles del campo "last_edited" apenas representan un 10.35% dentro de todo el DataFrame por lo cual este campo en su conjunto se considerará presindible

In [ ]:
# Se crea una variable que contiene los valores unicos del campo 'helpful'
valores_helpful = df_reviews['helpful'].unique() 
valores_helpful

In [ ]:
# Se crea otra variable para contar cuantos valores hay que no sean "No ratings yet"
cant_helpful = df_reviews[df_reviews['helpful'] != 'No ratings yet'].count()['helpful']
cant_helpful

In [ ]:
# Se hace una operación matematica para determinar que porcentaje representa los valores de "helpful" completos dentro del conjunto del DataFrame
print(f'La cantidad de valores que no son "No ratings yet" vacios en el campo "helpful" representan un {round((cant_helpful/total_filas_df)*100, 2)}%')

Ahora que se posee mas información sobre el campo "helpful" se puede determinar que es presindible no solo por que su información relevante represente un poco menos del 50%, sino también por que esta información 'útil' seria dificil de cuantificar y llevar a escala además en los requerimientos se indica que el campo mas relevante para el objetivo es "review" por lo cual este campo tambien se presindé.

In [ ]:
# Se crea una variable que contiene los valores unicos del campo 'recommend'
valores_recommend = df_reviews['recommend'].unique() 
valores_recommend

Con el proceso anterior se cerciora que el campo "recommend" contiene información relevante en todas las filas del DataFrame

In [ ]:
# Se cambia el tipo de dato de la variable 'recommend'
df_reviews['recommend'] = df_reviews['recommend'].astype(bool)
df_reviews.info()

#### Fechas

In [ ]:
# Se elimina la cadena 'Posted' de todos las filas del campo "posted"
df_reviews['posted'] = df_reviews['posted'].str.replace('Posted', '')
df_reviews.head(2)

In [ ]:
# Se crea un nuevo campo llamado "Date" donde se almacenara el contenido de "posted" convertido a formato datetime
df_reviews['Date'] = pd.to_datetime(df_reviews['posted'], dayfirst=True, errors='coerce') # Con estos atributos se logra primero que se reconozca adecuadamente el formato de la fecha en el campo "posted" y segundo que lo que no se reconozca como datetime se convierta en una NaT "Not a Time"
df_reviews.head(2)

In [ ]:
# Se crea una variable para contabilizar la cantidad NaT's que hay en el nuevo campo "Date" del DataFrame
cant_Not_Date = df_reviews['Date'].isna().sum()
cant_Not_Date

In [ ]:
# Se crea un nuevo campor llamado "Year" donde se almacenara solamente el año de la review
df_reviews['Year'] = pd.to_datetime(df_reviews['Date']).dt.year
df_reviews.head(2)

Debido a que se cuenta con valores NaT en el campo "Date" el output del campo "Year" esta en formato float por lo cual se van a tranformar a formato "int"

In [ ]:
# Se cuenta la cantidad de frecuencia de valores unicos que posee el campo "Year"
fecuencia_valores_Year = df_reviews['Year'].value_counts()
fecuencia_valores_Year

In [ ]:
# Se crea una variable para contabilizar la cantidad NaT's que hay en el nuevo campo "Year" del DataFrame
cant_Not_Year = df_reviews['Year'].isna().sum()
cant_Not_Year

Con las tranformaciones anteriores se puede establecer que la cantidad de filas donde faltan unicamente el año en el campo "posted" son 10119 y es por esta razón que hay nulos al momento de transformalos a formatos de fecha

In [ ]:
# Se reemplaza los valores NaT del campo "Year" por el año 2016
df_reviews['Year'].fillna(2016, inplace=True)
cant_Not_Year = df_reviews['Year'].isna().sum() # Se vuelve a contar los valores Nulos del campo
cant_Not_Year

In [ ]:
# Se cuenta la cantidad de frecuencia de valores unicos que posee el campo "Year" nuevamente
fecuencia_valores_Year = df_reviews['Year'].value_counts()
fecuencia_valores_Year

Esta decisión se justifica en un análisis externo de sobre Steam en la cual se encuentra que las reviews que no llevan el año en la fecha de posteo es por que fueron realizadas en el año en curso de esta manera como el último año que se tenía registro en el campo "posted" era el "2015" se asume que el año en curso es el "2016" y este pasa a reemplazar los valores Nulos que se tenian en estos campos de fechas

In [ ]:
# Se transforma los datos del campo "Year" de tipo float a tipo int
df_reviews['Year'] = df_reviews['Year'].astype(int)
df_reviews.head(2)

In [ ]:
# Se elimina el punto '.' de todos las filas del campo "posted"
df_reviews['posted'] = df_reviews['posted'].str.replace('.', '')
df_reviews.head(2)

Se procede a modificar el campo "posted" para poder añadir el año "2016" donde no lo tiene y de esta manera poder completar todos los valores del campo "Date"

In [ ]:
patron_4_digitos = r'(\d{4})$' # Se crea una variable para identificar los valores donde no haya un patrón de cuatro dígitos consecutivos
df_reviews.loc[~df_reviews['posted'].str.contains(patron_4_digitos, regex=True), 'posted'] += ', 2016' # Se crea una máscara booleana que es True para las filas donde no hay un patrón de 4 números consecutivos al final

El siguiente paso es hacer nuevamente el campo "Date" pero esta vez  se completaran todos los campos así no quedando valores Nulos

In [ ]:
# Se crea elimina el anterior campo "Date" y se transforma nuevamente
df_reviews = df_reviews.drop(columns='Date')
df_reviews['Date'] = pd.to_datetime(df_reviews['posted'], dayfirst=True, errors='coerce') # Con estos atributos se logra primero que se reconozca adecuadamente el formato de la fecha en el campo "posted" y segundo que lo que no se reconozca como datetime se convierta en una NaT "Not a Time"

In [ ]:
# Se nuevamente la variable para contabilizar la cantidad NaT's que hay en el nuevo campo "Date" del DataFrame
cant_Not_Date = df_reviews['Date'].isna().sum()
cant_Not_Date

Esta vez ya no hay valores Nulos en los campos de fechas

#### Eliminar campos no relevantes

In [ ]:
# Se crea una lista con el nombre de los campos que se van a eliminar del DataFrame
campos_out = ['user_url','reviews','funny','posted','last_edited','helpful']

In [ ]:
df_reviews = df_reviews.drop(columns=campos_out)
df_reviews.head(2)

En este punto que ya se tienen los campos relevantes se procede a normalizar los nombre de estos

In [ ]:
df_reviews = df_reviews.rename(columns={'user_id': 'User_Id'})
df_reviews = df_reviews.rename(columns={'item_id': 'Item_Id'})
df_reviews = df_reviews.rename(columns={'recommend': 'Recommend'})
df_reviews = df_reviews.rename(columns={'review': 'Review'})
df_reviews.head(2)

#### Se crea la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP

Se importa la libreria nltk y se descarga el lexicon 'vader_lexicon' necesario para realizar el análisis de sentimientos

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Se crea la función que analizara los valores de la columna "Review" para darles un valor númerico según el análisis de sentimientos

In [ ]:
# Se inicializa el analizador de sentimientos 
AS = SentimentIntensityAnalyzer()

# '0' si es malo 
# '1' si es neutral 
# '2' si es positivo

# Funcion para asignar valores de acuerdo a la escala
def puntaje_sentimiento(texto): # La variable de entrada de mi función seran los valores en formato str de la columna "Review"
    if pd.isnull(texto) or texto == '': # Evalua si el valor es un Nulo o un string vacío
        return 1 # Si es así retorna 1 que es un valor neutral
    elif isinstance(texto, str): # Verifica que el "texto" se una instancia del tipo str
        sentimiento = AS.polarity_scores(texto) # Se utiliza el analizador de sentimientos 'AS' para obtener un diccionario de puntajes de polarida y la función polarity_scores devuelve un diccionario que contiene puntajes para la polaridad positiva, negativa, neutra y un puntaje compuesto que resume la polaridad general del texto
        puntaje_compuesto = sentimiento['compound'] # Se extrae el puntaje compuesto del diccionario de sentimientos
        if puntaje_compuesto >= -0.05:
            return 2 # es positivo
        elif puntaje_compuesto <= -0.05:
            return 0 # es malo
        else:
            return 1 # es neutral
    else:
        return 1 # Si no cumple ninguna condición es neutral

In [ ]:
# Se convierte la columna 'Review' a formato str
df_reviews['Review'] = df_reviews['Review'].astype(str)

# Se aplica la función puntaje_sentimiento a la columna 'Review' creando una nueva columna llamada 'Sentiment_Analysis' que contendra los puntajes de este análisis
df_reviews['Sentiment_Analysis'] = df_reviews['Review'].apply(puntaje_sentimiento)
df_reviews.head(2)

# Tardo 20.7 seg

In [ ]:
# Se verifica que los únicos valores del campo sean cadenas que coincidan con años
fecuencia_valores_reviews = df_reviews['Sentiment_Analysis'].value_counts()
fecuencia_valores_reviews

Para este punto ya no es necesaria la columna "Review" y tal como se pide en las consignas esta ya cumplió su función y fue reemplazada por "Sentiment_Analysis" por lo caul va a ser eliminada

In [ ]:
df_reviews = df_reviews.drop(columns=['Review']) # Se elimina el campo 'Review'
df_reviews.head(2)

In [ ]:
df_reviews

### Se guarda el DataFrame en formatos mas ligeros

In [ ]:
df_reviews.to_csv('user_reviews.csv', index=False) # Se guarda el DataFrame en formato CSV

In [ ]:
# Se convierte el archivo CSV a parquet bajo la compresión "gzip"
ruta_csv_reviews = 'E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\PI_ML_OPS_STEAM_DSFT17\\user_reviews.csv' # Se crea una variable con la ruta del archivo CSV
df_reviews_temp = pd.read_csv(ruta_csv_reviews) # Se lee ese CSV en un nuevo DataFrame temporal para segurar como estan los datos
df_reviews_temp

Se guarada el DataFrame temporal en formato comprimido gzip

In [ ]:
df_reviews_temp.to_csv('user_reviews.gzip', compression='gzip', index=False)

In [ ]:
# Se verifica que el archivo gzip comprimido pueda ser leido de manera efectiva y que no hayan problemas en los datos que este arroja
df_reviews_parquet = pd.read_csv('user_reviews.gzip', compression='gzip')
df_reviews_parquet

In [ ]:
df_reviews_parquet.info()

## Data Set australian_users_items.json

In [ ]:
# Se inicializa una lista vacia donde alamcenare los datso del archivo .JSON
filas = []
# Se abre el archivo .JSON con el modulo 'with'
with open(ruta_items, 'r', encoding='utf-8') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

In [ ]:
# Se inicializa el DataFrame a partir de la lista donde se extrajo el archivo .JSON
df_items = pd.DataFrame(filas)
df_items

Se evidencia que la columna "items" contiene listas como dato así que se tiene que proceder a expandirla

In [ ]:
df_items = df_items.explode('items').reset_index() # Se redefine el DatFrame con una descomposición del campo para desanidarla y se reinicia el conteo del índice
df_items = df_items.drop(columns='index') # Se elimina el campo 'index'
df_items = pd.concat([df_items, pd.json_normalize(df_items['items'])], axis=1) # Se normalizan los datos JSON de la columna 'reviews' aplanando los datos, convirtiendolos en un formato tabular y concatenando de manera horizontal el DatFrame original con el del contendio de 'reviews' 
df_items.head(2)

### Se empieza a analizar y transformar los datos del DataFrame

In [ ]:
# Se verifica la información de los datos que contiene el DataFrame
df_items.info()

NULOS

In [ ]:
# Conteo de Nulos
nulos = df_items.isna().sum()
nulos

In [ ]:
# Se hace una operación matemática para determinar que poorcentaje de todo el DataSet representan esos valores Nulos
total_filas_df = 5170015
cant_nulos = 16806
print(f'La cantidad de valores nulos en el total del DataFrame es de {round(((cant_nulos/total_filas_df)*100), 2)}%')

Gracias a la operación anterior se determina que la cantidad de datos Nulos en el conjunto total del DataFrame representa una cantidad muy infima de los datos por lo cual se decide a ser eliminados

In [ ]:
# Se eliminan los valores Nulos
df_items = df_items.dropna()

In [ ]:
# Nuevamente se ha un conteo de Nulos
nulos = df_items.isna().sum()
nulos

In [ ]:
# Se verifica nuevamente la información del DataFrame
df_items.info()

#### Eliminar campos no relevantes

In [ ]:
# Se procede a eliminar campos no relevantes hasta este punto
df_items.drop(['user_url','items','playtime_2weeks'], axis=1, inplace=True)

Se considera el campo "playtime_2weeks" irrelevante ya que en las funciones que se especifican en la consigna solo piden las horas

In [ ]:
df_items.shape # Se visualiza las dimensiones del DataFrame

Debido al tamaño del DataFrame se considera eliminar las filas cuyo valor en "playtime_forever" sea igual a 0 ya que esto significa que el juego no se jugó por lo cual no sería considerado para valorar cada juego lo cual también necesario para las consignas osea las horas igual a 0 convierte las filas en irrelevantes

In [ ]:
df_items = df_items[df_items['playtime_forever'] != 0.0] # Con este código reasigno el DatFrame sin las filas cuyos valores en 'playtime_forever' sea igual a 0.0 horas
df_items.shape

In [ ]:
# Se hace una operación para determinar en que porcentaje se redujo el tamaño del DataFrame
print(f'El tamaño del DataFrame se redujo en un {100 - (round(((3285246/5153209)*100), 2))}%')

#### Transformación de datos

##### A traves de este [link](https://developer.valvesoftware.com/wiki/Steam_Web_API) se puede observar que se menciona lo siguiente "playtime_forever The total number of minutes played "on record", since Steam began tracking total playtime in early 2009." por lo cual se intuye que este campo representa minutos pero ya que en las consignas se piden horas para las funciones de consulta se procede a tranformar a horas jugadas

In [ ]:
# Se crea una función que convierte los minutos a horas y utilizando la función math.ceil(x) redondea hacia arriba el valor para tener valores enteros en horas
import math

def convertir_a_horas(aplicar):
    # Función para convertir minutos a horas y redondear hacia arriba
    
    # Verificar si la columna ya está en formato float, si no, convertir
    aplicar = aplicar.astype(float)
    
    # Convertir minutos a horas y redondear hacia arriba
    horas = aplicar / 60
    horas_redondeadas = horas.apply(lambda x: math.ceil(x))
    
    return horas_redondeadas

In [ ]:
df_items['playtime_forever'] = convertir_a_horas(df_items['playtime_forever'])
df_items.head(2)

##### Se procede a cambiar el tipo de dato del campo "item_id" por tipo int

In [ ]:
# Se crea una variable que contiene los valores unicos del campo 'item_id' para evalular que solo contenga numeros
valores_item_id = df_items['item_id'].unique() 
valores_item_id

In [ ]:
# Ahora que se sabe que solo contiene valores númericos se procede a realizar el cambio de tipo de dato
df_items['item_id'] = df_items['item_id'].astype(int)

In [ ]:
df_items.info() # Se verifica que haya cambiado el tipo de dato del campo 'item_id'

In [ ]:
df_items.head(2)

En este punto que ya se tienen los campos relevantes se procede a normalizar los nombre de estos

In [ ]:
df_items = df_items.rename(columns={'user_id': 'User_Id'})
df_items = df_items.rename(columns={'items_count': 'Items_Count'})
df_items = df_items.rename(columns={'steam_id': 'Steam_Id'})
df_items = df_items.rename(columns={'item_id': 'Item_Id'})
df_items = df_items.rename(columns={'item_name': 'Item_Name'})
df_items = df_items.rename(columns={'playtime_forever': 'Playtime_Forever_Hours'})
df_items.head(2)

### Se guarda el DataFrame en formatos mas ligeros

In [ ]:
df_items.to_csv('user_items.csv', index=False) # Se guarda el DataFrame en formato CSV)

In [ ]:
# Se convierte el archivo CSV a parquet bajo la compresión "gzip"
ruta_csv_items = 'E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\PI_ML_OPS_STEAM_DSFT17\\user_items.csv' # Se crea una variable con la ruta del archivo CSV
df_items_temp = pd.read_csv(ruta_csv_items) # Se lee ese CSV en un nuevo DataFrame temporal para segurar como estan los datos
df_items_temp

Se guarada el DataFrame temporal en formato comprimido gzip

In [ ]:
df_items_temp.to_csv('user_items.gzip', compression='gzip', index=False)

In [ ]:
# Se verifica que el archivo gzip comprimido pueda ser leido de manera efectiva y que no hayan problemas en los datos que este arroja
df_items_parquet = pd.read_csv('user_items.gzip', compression='gzip')
df_items_parquet

In [ ]:
df_items_parquet.info()

## Data Set output_steam_games.json

In [ ]:
# Se inicializa una lista vacia donde alamcenare los datso del archivo .JSON
filas = []
# Se abre el archivo .JSON con el modulo 'with'
with open(ruta_games, 'r', encoding='utf-8') as archivo:
    for line in archivo.readlines():
        filas.append(json.loads(line))

In [ ]:
# Se inicializa el DataFrame a partir de la lista donde se extrajo el archivo .JSON
df_games = pd.DataFrame(filas)
df_games

### Se empieza a analizar y transformar los datos del DataFrame

In [ ]:
# Se verifica la información de los datos que contiene el DataFrame
df_games.info()

#### Eliminar campos no relevantes

In [ ]:
# Se crea una lista con el nombre de los campos que se van a eliminar del DataFrame
campos_eliminar = ['app_name','url','reviews_url','specs','price','early_access']

In [ ]:
df_games = df_games.drop(columns=campos_eliminar)
df_games.head(2)

##### Se decidio eliminar estos campos debido a diversas razones que van desde que tenian información poco relevante, información que se cumplia con la de otro campo o que no se considera necesaria para cumplir con las consignas que se piden

NULOS

In [ ]:
# Conteo de Nulos
nulos = df_games.isna().sum()
nulos

Se hace notorio que hay filas que estan completamente llenas de NaN's así que se procedera a eliminar esas primero

In [ ]:
df_games = df_games.dropna(how='all') # Se le indica que elimine solo las filas que contienen NaN en todos los campos

In [ ]:
# Se vuelve a contar los Nulos
nulos = df_games.isna().sum()
nulos

In [ ]:
# Se revisa el nuevo tamaño del DataFrame para ver en cuanto se redujo su contenido
df_games.shape

In [ ]:
print(f'Eliminando solamente las filas completas de valores Nulos se redujo el Data Set en un {100 - (round(((32134/120445)*100) ,2))}%')

Se elimina la fila donde existe un valor Nulo en el campo "id"

In [ ]:
df_games.dropna(subset=["id"], inplace=True)

##### Siguiendo las consignas de las funciónes de colsulta se puede determinar que los campos "genres" y "developer" son crusiales por lo cual se procedera a eliminar reemplazar sus datos con los campos que se consideran similares para darles una caracaterestica y las filas donde estos tengan valores Nulos en ambos campos serán eliminados

In [ ]:
def reemplazar_nulos_genres(df):
    # Función para reemplazar valores nulos en 'genres' con valores de 'tags'
    # y eliminar filas donde ambos son nulos
    
    # Verificar si hay valores no nulos en 'tags'
    mask = df['tags'].notnull()
    
    # Reemplazar valores nulos en 'genres' con valores de 'tags'
    df.loc[mask, 'genres'] = df.loc[mask, 'tags']
    
    # Eliminar filas donde tanto 'genres' como 'tags' son nulos
    df.dropna(subset=['genres', 'tags'], how='all', inplace=True)
    
    return df

In [ ]:
df_games = reemplazar_nulos_genres(df_games) # Se ejecuta la función en el DataFrame

In [ ]:
# Se vuelve a contar los Nulos
nulos = df_games.isna().sum()
nulos

In [ ]:
def reemplazar_nulos_developer(df):
    # Función para reemplazar valores nulos en 'developer' con valores de 'tags'
    # y eliminar filas donde ambos son nulos
    
    # Verificar si hay valores no nulos en 'publisher'
    mask = df['publisher'].notnull()
    
    # Reemplazar valores nulos en 'developer' con valores de 'publisher'
    df.loc[mask, 'developer'] = df.loc[mask, 'publisher']
    
    # Eliminar filas donde tanto 'developer' como 'publisher' son nulos
    df.dropna(subset=['developer', 'publisher'], how='all', inplace=True)
    
    return df

In [ ]:
df_games = reemplazar_nulos_developer(df_games) # Se ejecuta la función en el DataFrame

In [ ]:
# Se vuelve a contar los Nulos
nulos = df_games.isna().sum()
nulos

In [ ]:
df_games.shape # Se analiza el número de filas que tiene el DataSet hasta el momento

En base a lo anterior se considera que la cantidad de valores Nulos del campo "release_date" son infimos dentro del conjunto del DataFrame por lo cual se procede a ser eliminados, es de aclarar que la información de este campo tambien es necesaria para las consginas

In [ ]:
df_games.dropna(subset=["release_date"], inplace=True) # Se eliminan los nulos del campo 'release_date'
df_games.shape

In [ ]:
# Se revisa cuantos valores nulos se tiene hasta el momento
nulos = df_games.isna().sum()
nulos

La cantidad de Nulos que quedan en el campo "publisher" es considerable además de ser un campo que podría no ser presendible para las funciones se decide dejarlo y reemplazar los valores Nulos por un string que indique "No Data" de esta manera "N/D" y lo mismo para el campo "tags"

In [ ]:
df_games.info() 

Se observa que el tipo de dato de todos los campos esta en "str"

In [ ]:
# Se reemplaza los valores Nulos del campo 'tags' por una la siguiente lista '[N/D]'
df_games['tags'] = df_games['tags'].fillna('[N/D]')

In [ ]:
# Se verifica que ya no hayan valores Nulos en el campo 'tags'
nulos = df_games.isna().sum()
nulos

In [ ]:
# Se reemplaza los valores Nulos del campo 'publisher' por una el siguiente string 'N/D'
df_games['publisher'] = df_games['publisher'].fillna('N/D')

In [ ]:
# Se verifica que ya no hayan valores Nulos en el campo 'publisher'
nulos = df_games.isna().sum()
nulos

Hasta este punto ya no hay valores Nulos en el Data Frame

##### Se identifica una complejidad en el campo "release_date" por lo cual se procede a ser tranformado

In [ ]:
# Se realiza un filtrado de los valores que no coinciden con el patrón de fecha
import re
patron = re.compile(r'^\d{4}-\d{2}-\d{2}$')
filas_no_patron = df_games[~df_games['release_date'].astype(str).str.match(patron)]
filas_no_patron

##### Se descubre que hay una cantidad de filas cuyos datos de "release_date" son muy variados pero se identifica un patrón en ellos haciendo posible que se pueda salvar algunas filas

Se crea una función para extraer los últimos 4 digitos de los valores si los hay y los suma a un string para poder darle el formato que se necesita las fechas

In [ ]:
def convertir_fecha(fecha_str):
    # Función para convertir expresiones de fecha en formato de cadena a "YYYY-MM-DD"
    
    # Buscar los últimos cuatro dígitos en la cadena
    ultimos_cuatro_digitos = ''.join(filter(str.isdigit, fecha_str))[-4:]
    
    # Concatenar con la cadena "-01-01"
    fecha_resultante = ultimos_cuatro_digitos + "-01-01"
    
    return fecha_resultante

In [ ]:
# Se crea un nuevo Data Frame para guardar el resultado de la función
df_fechas_corregidas = pd.DataFrame()
df_fechas_corregidas['release_date'] = filas_no_patron['release_date'].apply(convertir_fecha)

In [ ]:
df_fechas_corregidas

In [ ]:
# Se elimina las filas que no fueron transformadas de la manera deseada
df_fechas_corregidas = df_fechas_corregidas[df_fechas_corregidas['release_date'] != "-01-01"]
df_fechas_corregidas

Se aplican los cambios de las fechas corregidas al Data Frame orginal

In [ ]:
df_games.update(df_fechas_corregidas)
df_games.head(2)

Nuevamente se buscan las filas donde el campo "release_date" no coincida con el formato que se quiere

In [ ]:
filas_no_patron = df_games[~df_games['release_date'].astype(str).str.match(patron)]
filas_no_patron

In [ ]:
# Se eliminan las filas que no coinciden con el patron de fecha de 'release_date'
df_games = df_games.drop(filas_no_patron.index)
df_games.shape

In [ ]:
# Se verifica que no hayan valores mas largos como valor en'realease_rate' que no se puedan convertir a una fecha
longitud_no_deseada = df_games[df_games['release_date'].astype(str).apply(len) != 10]
longitud_no_deseada

In [ ]:
# Se eliminan los últimos 6 dijitos del campo 'release_date' para que solamente quede los dijitos del año
df_games['release_date'] = df_games['release_date'].astype(str).str[:4]
df_games.head()

In [ ]:
# Se verifica que los únicos valores del campo sean cadenas que coincidan con años
fecuencia_valores_date = df_games['release_date'].unique()
fecuencia_valores_date

Se identifican unos valores que no coinciden con un año que puedieron aparecer al momento de rescatar algunas filas

In [ ]:
cantidad_valores_date = df_games['release_date'].value_counts()
cantidad_valores_date

In [ ]:
# Se guaradan esos valores en una lista
no_años = ['6441', '0181', '0111', '0174', '0171']

# Se filtra por los valores que no coincidan con esa lista
df_games = df_games[~df_games['release_date'].str[:4].isin(no_años)]

In [ ]:
# Se verifica nuevamente que los únicos valores del campo sean cadenas que coincidan con años
fecuencia_valores_date = df_games['release_date'].unique()
fecuencia_valores_date

Se reinicia el conteo del index del Data Frame

In [ ]:
df_games = df_games.reset_index()
df_games = df_games.drop(columns='index')

Se procede a Normalizar los nombres de los campos

In [ ]:
df_games = df_games.rename(columns={'publisher': 'Publisher'})
df_games = df_games.rename(columns={'genres': 'Genres'})
df_games = df_games.rename(columns={'title': 'Title'})
df_games = df_games.rename(columns={'release_date': 'Year'})
df_games = df_games.rename(columns={'tags': 'Tags'})
df_games = df_games.rename(columns={'id': 'Item_Id'})
df_games = df_games.rename(columns={'developer': 'Developer'})
df_games.head(2)

#### Se transforma los tipos de datos de los valores 

In [ ]:
# Se cambia el tipo de dato del campo 'Release_Date' por tipo de dato Date Time
df_games['Year'] = df_games['Year'].astype(int)
df_games.head(2)

In [ ]:
# Se transforma los datos del campo 'Item_Id' de tipo str a tipo int
df_games['Item_Id'] = df_games['Item_Id'].astype(int)
df_games.head(2)

In [ ]:
# Se transforma los datos de los campos 'Genres' y 'Tags' de tipo str a tipo str
df_games['Genres'] = df_games['Genres'].astype(str)
df_games['Tags'] = df_games['Tags'].astype(str)
df_games.head(2)

In [ ]:
# Se verifica estos cambios de tipos de dato
df_games.info()

In [ ]:
# Se elimina los corchetes del principio y el final de las cadenas
df_games['Genres'] = df_games['Genres'].str.strip('[]')
df_games['Tags'] = df_games['Tags'].str.strip('[]')
df_games.head(2)

In [ ]:
# Se elimina la la comilla "'" de todos las filas de los campos  'Genres' y 'Tags'
df_games['Genres'] = df_games['Genres'].str.replace("'", '')
df_games['Tags'] = df_games['Tags'].str.replace("'", '')
df_games.head(2)

In [ ]:
# Se crea una variable para contar cuantos valores hay que sean strings ("") en el campo 'Generes'
cant_generes = df_games[df_games['Genres'] == ''].count()['Genres']
cant_generes

In [ ]:
# Se crea una variable para contar cuantos valores hay que sean strings ("") en el campo 'Tags'
cant_tags = df_games[df_games['Tags'] == ''].count()['Tags']
cant_tags

De esta manera se verifica que ya no hay filas con datos irelevantes hasta el momento

##### Se decide tomar como relevante el contenido del campo "Tags" para las consultas sobre los generos por lo cual se unira al campo "Genres" creando un nuevo campo llamado "Genres_Plus" donde solo aparezcan los valores únicos resultantes 

In [ ]:
df_games['Genres_Plus'] = (df_games['Genres'].str.split(', ') + df_games['Tags'].str.split(', ')).apply(set).apply(list).apply(', '.join)
df_games.head()

Ahora que se tiene el nuevo campo se procede a eliminar los originales y a normalizar el nuevo

In [ ]:
# Se eliminan las columnas orginales
df_games = df_games.drop(columns='Genres')
df_games = df_games.drop(columns='Tags')
df_games.head(2)

In [ ]:
# Se le cambia el nombre al campo nuevo
df_games = df_games.rename(columns={'Genres_Plus': 'Genres'})
df_games.head(2)

Ya que en algun punto se le asigo el valor "[N/D]" a los Nulos del campo "tags" ahora se puede proceder a ser eliminados ya que son innecesarios

In [ ]:
# Se visualiza las filas donde existe ese valor en el campo 'Genres'
df_filtrado = df_games[df_games['Genres'].str.contains('N/D')]
df_filtrado

In [ ]:
# Se elimina el valor "N/D" de todos las filas del campo 'Genres
df_games['Genres'] = df_games['Genres'].str.replace("N/D", '')

In [ ]:
# Se verifica que el valor ya no exista en el campo 'Genres'
df_filtrado = df_games[df_games['Genres'].str.contains('N/D')]
df_filtrado

Para una mejor lectura del Data Frame se decide cambiar la posición del campo "Publisher"

In [ ]:
columnas = list(df_games.columns)
columnas[0], columnas[3] = columnas[3], columnas[0]
df_games = df_games[columnas]
df_games.head()

### Se guarda el DataFrame en formatos mas ligeros

In [ ]:
df_games.to_csv('user_games.csv', index=False) # Se guarda el DataFrame en formato CSV)

In [ ]:
# Se convierte el archivo CSV a parquet bajo la compresión "gzip"
ruta_csv_items = 'E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\PI_ML_OPS_STEAM_DSFT17\\user_games.csv' # Se crea una variable con la ruta del archivo CSV
df_games_temp = pd.read_csv(ruta_csv_items) # Se lee ese CSV en un nuevo DataFrame temporal para segurar como estan los datos
df_games_temp

Se guarada el DataFrame temporal en formato comprimido gzip

In [ ]:
df_games_temp.to_csv('user_games.gzip', compression='gzip', index=False)

In [ ]:
# Se verifica que el archivo gzip comprimido pueda ser leido de manera efectiva y que no hayan problemas en los datos que este arroja
df_games_parquet = pd.read_csv('user_games.gzip', compression='gzip')
df_games_parquet

In [ ]:
df_games_parquet.info()